In [ ]:
import sys
!{sys.executable} -m pip install librosa
!{sys.executable} -m pip install jupyter_helpers

In [1]:
# Load various imports 
import pandas as pd
import numpy as np
import os
import glob
import wave
import librosa
import librosa.display

In [2]:
import numpy as np
max_pad_len = 1200
def extract_features2(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [3]:
fulldatasetpath = 'F:\Essential Files\Coding Projects\senior design 2\wav_files'

metadata = pd.read_csv('F:\Essential Files\Coding Projects\senior design 2\metadata.csv')

features2 = []

for index, row in metadata.iterrows():
    name = "\\"
    name = name + str(row["file"])
    file_name = os.path.join(fulldatasetpath + name)
    
    data = extract_features2(file_name)
    animal = str(row["animal"])
    features2.append([data, animal])

In [4]:
featuresdf = pd.DataFrame(features2, columns=['feature','animal'])

In [ ]:
!{sys.executable} -m pip install tensorflow

In [ ]:
!{sys.executable} -m pip install keras-utils

In [ ]:
pip install --ignore-installed --upgrade tensorflow

In [5]:
import tensorflow as tf

In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.animal.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

Using TensorFlow backend.


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 1200
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

In [8]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [9]:

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 39, 1199, 16)      80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 599, 16)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 599, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 598, 32)       2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 299, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 299, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 298, 64)       

In [10]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

#num_epochs = 12
#num_batch_size = 128

num_epochs = 72
num_batch_size = 900

checkpointer = ModelCheckpoint(filepath='F:\Essential Files\Coding Projects\senior design 2\weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 224 samples, validate on 57 samples
Epoch 1/72
224/224 [==============================] - 5s 25ms/step - loss: 1.8399 - accuracy: 0.2991 - val_loss: 1.0456 - val_accuracy: 0.4035

Epoch 00001: val_loss improved from inf to 1.04556, saving model to F:\Essential Files\Coding Projects\senior design 2\weights.best.basic_mlp.hdf5
Epoch 2/72
224/224 [==============================] - 5s 21ms/step - loss: 1.4659 - accuracy: 0.3304 - val_loss: 1.1512 - val_accuracy: 0.3509

Epoch 00002: val_loss did not improve from 1.04556
Epoch 3/72
224/224 [==============================] - 5s 21ms/step - loss: 1.8910 - accuracy: 0.4241 - val_loss: 1.0690 - val_accuracy: 0.3860

Epoch 00003: val_loss did not improve from 1.04556
Epoch 4/72
224/224 [==============================] - 5s 23ms/step - loss: 1.6472 - accuracy: 0.3839 - val_loss: 1.0765 - val_accuracy: 0.4035

Epoch 00004: val_loss did not improve from 1.04556
Epoch 5/72
224/224 [==============================] - 5s 21ms/step - loss: 1.68

In [11]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9017857313156128
Testing Accuracy:  0.9122806787490845


In [12]:
def print_prediction(file_name):
    prediction_feature = extract_features2(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [14]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\dog_barking_113.wav' 
print_prediction(filename)

The predicted class is: cat 

cat 		 :  0.42054960131645202636718750000000
chicken 		 :  0.42022961378097534179687500000000
dog 		 :  0.15922069549560546875000000000000


In [15]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\cat_97.wav' 
print_prediction(filename)

The predicted class is: cat 

cat 		 :  0.77720570564270019531250000000000
chicken 		 :  0.04165717586874961853027343750000
dog 		 :  0.18113709986209869384765625000000


In [16]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\dog_barking_114.wav' 
print_prediction(filename)

The predicted class is: chicken 

cat 		 :  0.24113853275775909423828125000000
chicken 		 :  0.40302091836929321289062500000000
dog 		 :  0.35584050416946411132812500000000


In [17]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\cat_96.wav' 
print_prediction(filename)

The predicted class is: cat 

cat 		 :  0.73466926813125610351562500000000
chicken 		 :  0.05041634291410446166992187500000
dog 		 :  0.21491438150405883789062500000000


In [18]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\dog_barking_70.wav' 
print_prediction(filename)

The predicted class is: dog 

cat 		 :  0.02595599554479122161865234375000
chicken 		 :  0.00586052192375063896179199218750
dog 		 :  0.96818351745605468750000000000000


In [19]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\\rooster_10.wav' 
print_prediction(filename)

The predicted class is: chicken 

cat 		 :  0.20278236269950866699218750000000
chicken 		 :  0.59909230470657348632812500000000
dog 		 :  0.19812534749507904052734375000000


In [20]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\\rooster_3.wav' 
print_prediction(filename)

The predicted class is: chicken 

cat 		 :  0.20410080254077911376953125000000
chicken 		 :  0.61332899332046508789062500000000
dog 		 :  0.18257024884223937988281250000000
